In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_json("data/hotels_processed.json")

In [3]:
df= df.drop(columns="place_name")

In [4]:
def fill_distance(row):
    if pd.isna(row['distance_from_downtown_km']):
        if row['hotel_type'] in ['resort', 'luxury']:
            return np.random.uniform(8,20)
        else:
            return np.random.uniform(0.5,5)
    return row['distance_from_downtown_km']


In [5]:
df['distance_from_downtown_km'] = df.apply(fill_distance,axis=1)

In [6]:
df['price_per_night'] = df['price_per_night'].fillna(df['price_per_night'].median())

In [7]:
df = df.drop(columns=['stars'])

In [8]:
df['rating'] = df['rating'].fillna(df['rating'].mean())

In [9]:
encode = MultiLabelBinarizer()
tags = encode.fit_transform(df['amenities'])
tag_df=pd.DataFrame(tags,columns=encode.classes_)
df=pd.concat([df,tag_df],axis=1)

In [10]:
encode = OneHotEncoder(sparse_output=False)
typ = encode.fit_transform(df[['hotel_type']])
typ_df = pd.DataFrame(typ,columns=encode.get_feature_names_out(['hotel_type']))
df = pd.concat([df,typ_df],axis=1)

In [11]:
df= df.drop(columns=["hotel_type","amenities"])

In [12]:
df = df.set_index('hotel_id')

In [13]:

for col in df.columns:
    if df[col].dropna().isin([0, 1]).all():
        df[col] = df[col].astype('int64')


In [14]:
df.to_csv("data/hotels_preprocessed.csv",index=True)